In [1]:
import numpy as np
import librosa
import matplotlib.pyplot as plt
import librosa.display
from scipy.ndimage import maximum_filter, gaussian_filter
import hashlib

In [4]:
import hashlib

def generate_hashes(peaks, fan_value=5):
    """
    Generate landmark hashes from peaks.
    Each hash is made from (freq1, freq2, delta_time), anchored at time1.
    """
    hashes = []
    peaks = sorted(peaks)  # Sort by time

    for i in range(len(peaks)):
        t1, f1 = peaks[i]

        for j in range(1, fan_value + 1):
            if i + j < len(peaks):
                t2, f2 = peaks[i + j]

                delta_t = t2 - t1
                if 0 < delta_t <= 5.0:  # Limit max time delta for compact hashes
                    # Create hash string
                    hash_str = f"{int(f1)}|{int(f2)}|{int(delta_t * 100)}"
                    h = hashlib.sha1(hash_str.encode('utf-8')).hexdigest()[0:20]  # Shorten for space
                    hashes.append((h, t1))  # Store hash + anchor time

    return hashes



In [24]:
import json

with open('songs_meta.json', 'r', encoding='utf-8') as f:
    songsMeta = json.load(f)

hashIndex={}
for song in songsMeta.values():
    y, sr = librosa.load(song["location"], sr=None)

    # Step 2: Compute STFT
    n_fft = 2048
    hop_length = 512
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    S_mag = np.abs(S)

    # Step 3: Smooth spectrogram (optional, helps with noisy peaks)
    S_smooth = gaussian_filter(S_mag, sigma=1.0)

    # Step 4: 2D local maxima detection
    neighborhood_size = 20
    local_max = maximum_filter(S_smooth, size=neighborhood_size) == S_smooth

    # Step 5: Apply a threshold to keep only strong peaks
    threshold = np.percentile(S_smooth[local_max], 95)  # keep top 5%
    peak_mask = local_max & (S_smooth >= threshold)

    # Step 6: Get peak coordinates (freq_bin, time_bin)
    peak_indices = np.argwhere(peak_mask)

    # Step 7: Convert to time (sec) and frequency (Hz)
    peak_freqs = peak_indices[:, 0] * (sr / n_fft)
    peak_times = peak_indices[:, 1] * (hop_length / sr)

    # Combine for (time, freq) tuples
    constellation_map = list(zip(peak_times, peak_freqs))

    #get fingerprints
    fingerprint = generate_hashes(constellation_map)

    for h,t in fingerprint:
        if h not in hashIndex:
            hashIndex[h]=[(song["title"],t)]
        else:
            hashIndex[h].append((song["title"], t))

C:\Users\udayg\AppData\Local\Temp\ipykernel_16572\82423531.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(song["location"], sr=None)


In [25]:
def SearchSong(songLocation):
    """
    Search for a song by title and artist.
    Returns the fingerprints if found, else None.
    """
    y, sr = librosa.load(songLocation, sr=None)

    # Step 2: Compute STFT
    n_fft = 2048
    hop_length = 512
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    S_mag = np.abs(S)

    # Step 3: Smooth spectrogram (optional, helps with noisy peaks)
    S_smooth = gaussian_filter(S_mag, sigma=1.0)

    # Step 4: 2D local maxima detection
    neighborhood_size = 20
    local_max = maximum_filter(S_smooth, size=neighborhood_size) == S_smooth

    # Step 5: Apply a threshold to keep only strong peaks
    threshold = np.percentile(S_smooth[local_max], 95)  # keep top 5%
    peak_mask = local_max & (S_smooth >= threshold)

    # Step 6: Get peak coordinates (freq_bin, time_bin)
    peak_indices = np.argwhere(peak_mask)

    # Step 7: Convert to time (sec) and frequency (Hz)
    peak_freqs = peak_indices[:, 0] * (sr / n_fft)
    peak_times = peak_indices[:, 1] * (hop_length / sr)

    # Combine for (time, freq) tuples
    constellation_map = list(zip(peak_times, peak_freqs))

    #get fingerprints
    fingerprint = generate_hashes(constellation_map)
    score={
    }
    for h,t_query in fingerprint:
        if h in hashIndex:
            for (song,t_db) in hashIndex[h]:
                delta= round(t_db-t_query, 2)
                score[(song,delta)] = score.get((song,delta), 0) + 1
    best_match = None
    best_score = 0

    for (song, delta), count in score.items():
        if best_match is None or count > best_score:
            best_match = song
            best_score = count
    print(score)
    if best_match is not None and best_match in songsMeta:  # You can tweak this threshold
        print("Match found!")
        print(songsMeta[best_match])
        return songsMeta[best_match]
    else:
        print("No matching song found in the database.")
        return None


In [45]:
SearchSong("Testsongs\\RAWAL x Bharg x Encore ABJ - MAGAN (mp3cut.net) (1).mp3")

{}
No matching song found in the database.


[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg
[youtube:tab] PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg: Downloading webpage


[youtube:tab] PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: idk
[youtube:tab] PLwi5bnlaFJvgc7x1MPvV3h2KyU5vlVQKg page 1: Downloading API JSON
[youtube:tab] Playlist idk: Downloading 5 items of 5
[download] Downloading item 1 of 5
[youtube] Extracting URL: https://www.youtube.com/watch?v=p0kKahGOKe8
[youtube] p0kKahGOKe8: Downloading webpage
[youtube] p0kKahGOKe8: Downloading tv client config
[youtube] p0kKahGOKe8: Downloading tv player API JSON
[youtube] p0kKahGOKe8: Downloading ios player API JSON
[youtube] p0kKahGOKe8: Downloading m3u8 information
[info] p0kKahGOKe8: Downloading 1 format(s): 251
[download] Destination: songs\RAWAL x Bharg x Encore ABJ - MAGAN.webm
[download] 100% of    3.38MiB in 00:00:03 at 977.07KiB/s 
[ExtractAudio] Destination: songs\RAWAL x Bharg x Encore ABJ - MAGAN.mp3
Deleting original file songs\RAWAL x Bharg x Encore ABJ - MAGAN.webm (pass -k to keep)
[download] Downloading item 

         player = https://www.youtube.com/s/player/e12fbea4/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] 2k7E3u36gt4: Downloading m3u8 information
[info] Testing format 234
[info] 2k7E3u36gt4: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 46
[download] Destination: songs\Khoya Sitara - Dizlaw (Official music video) ｜ Azadi Records.mp4
[download] 100% of    3.88MiB in 00:00:11 at 354.20KiB/s                
[ExtractAudio] Destination: songs\Khoya Sitara - Dizlaw (Official music video) ｜ Azadi Records.mp3
Deleting original file songs\Khoya Sitara - Dizlaw (Official music video) ｜ Azadi Records.mp4 (pass -k to keep)
[download] Downloading item 3 of 5
[youtube] Extracting URL: https://www.youtube.com/watch?v=6Iox2oVm1sI
[youtube] 6Iox2oVm1sI: Downloading webpage
[youtube] 6Iox2oVm1sI: Downloading tv client config
[youtube] 6Iox2oVm1sI: Downloading tv player API JSON
[youtube] 6Iox2oVm1sI: Downloading ios player API JSON


         player = https://www.youtube.com/s/player/e12fbea4/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] 6Iox2oVm1sI: Downloading m3u8 information
[info] Testing format 234
[info] 6Iox2oVm1sI: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 50
[download] Destination: songs\Bharg & @ChaarDiwaari  - Roshni ｜ Nikamma.mp4
[download] 100% of    3.95MiB in 00:01:01 at 66.12KiB/s                  
[ExtractAudio] Destination: songs\Bharg & @ChaarDiwaari  - Roshni ｜ Nikamma.mp3
Deleting original file songs\Bharg & @ChaarDiwaari  - Roshni ｜ Nikamma.mp4 (pass -k to keep)
[download] Downloading item 4 of 5
[youtube] Extracting URL: https://www.youtube.com/watch?v=jj3AOSCEGp4
[youtube] jj3AOSCEGp4: Downloading webpage
[youtube] jj3AOSCEGp4: Downloading tv client config
[youtube] jj3AOSCEGp4: Downloading tv player API JSON
[youtube] jj3AOSCEGp4: Downloading ios player API JSON


         player = https://www.youtube.com/s/player/e12fbea4/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] jj3AOSCEGp4: Downloading m3u8 information
[info] Testing format 234
[info] jj3AOSCEGp4: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 41
[download] Destination: songs\Naksha.mp4
[download] 100% of    3.21MiB in 00:00:08 at 402.15KiB/s                
[ExtractAudio] Destination: songs\Naksha.mp3
Deleting original file songs\Naksha.mp4 (pass -k to keep)
[download] Downloading item 5 of 5
[youtube] Extracting URL: https://www.youtube.com/watch?v=6pQcD3xKAtA
[youtube] 6pQcD3xKAtA: Downloading webpage
[youtube] 6pQcD3xKAtA: Downloading tv client config
[youtube] 6pQcD3xKAtA: Downloading tv player API JSON
[youtube] 6pQcD3xKAtA: Downloading ios player API JSON


ERROR: [youtube] 6pQcD3xKAtA: Video unavailable. This video is not available


[download] Finished downloading playlist: idk


In [ ]:
import os
import json
import re
from yt_dlp import YoutubeDL

# Step 1: Create a valid filename function
def clean_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "", name)

# Step 2: Prepare metadata dictionary
songs_meta = {}

# Load existing metadata if present
meta_file = "songs_meta.json"
if os.path.exists(meta_file):
    with open(meta_file, "r", encoding="utf-8") as f:
        songs_meta = json.load(f)

# Step 3: Define output directory
output_dir = "songs"
os.makedirs(output_dir, exist_ok=True)

# Step 4: Define progress hook
def progress_hook(d):
    if d['status'] == 'finished':
        info = d['info_dict']
        title = info.get('title', 'Unknown Title')
        artist = info.get('uploader', 'Unknown Artist')
        video_id = info.get('id')
        ext = info.get('ext', 'mp3')

        safe_title = clean_filename(title)
        filename = f"{safe_title}.mp3"
        original_path = f"{video_id}.{ext}"
        final_path = os.path.join(output_dir, filename)

        if os.path.exists(original_path):
            os.rename(original_path, final_path)

        # Add or update song metadata
        songs_meta[safe_title] = {
            "title": title,
            "artist": artist,
            "location": final_path
        }

# Step 5: Define YT-DLP options
def get_yt_dlp_opts():
    return {
        'format': 'bestaudio/best',
        'ignoreerrors': True,
        'quiet': False,
        'noplaylist': False,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'progress_hooks': [progress_hook],
        'outtmpl': '%(id)s.%(ext)s',
    }

# Step 6: Start download
playlist_url = "https://youtube.com/playlist?list=PLwi5bnlaFJvjbnvBu4aLwuGkQH4F45nUO"

with YoutubeDL(get_yt_dlp_opts()) as ydl:
    ydl.download([playlist_url])

# Step 7: Save updated metadata
with open(meta_file, "w", encoding="utf-8") as f:
    json.dump(songs_meta, f, indent=4, ensure_ascii=False)


[youtube:tab] Extracting URL: https://youtube.com/playlist?list=PLwi5bnlaFJvjbnvBu4aLwuGkQH4F45nUO
[youtube:tab] PLwi5bnlaFJvjbnvBu4aLwuGkQH4F45nUO: Downloading webpage


[youtube:tab] PLwi5bnlaFJvjbnvBu4aLwuGkQH4F45nUO: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: Yellow
[youtube:tab] PLwi5bnlaFJvjbnvBu4aLwuGkQH4F45nUO page 1: Downloading API JSON
[youtube:tab] Playlist Yellow: Downloading 23 items of 23
[download] Downloading item 1 of 23
[youtube] Extracting URL: https://www.youtube.com/watch?v=jtycySTCRWI
[youtube] jtycySTCRWI: Downloading webpage
[youtube] jtycySTCRWI: Downloading tv client config
[youtube] jtycySTCRWI: Downloading tv player API JSON
[youtube] jtycySTCRWI: Downloading ios player API JSON
[youtube] jtycySTCRWI: Downloading player e12fbea4-main


         player = https://www.youtube.com/s/player/e12fbea4/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] jtycySTCRWI: Downloading m3u8 information
[info] Testing format 234
[info] jtycySTCRWI: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 12
[download] Destination: jtycySTCRWI.mp4
[download] 100% of  962.31KiB in 00:00:01 at 525.75KiB/s                


ERROR: WARNING: unable to obtain file audio codec with ffprobe


...

[download] Downloading item 23 of 23
[youtube] Extracting URL: https://www.youtube.com/watch?v=Wh-AgS7PKdM
[youtube] Wh-AgS7PKdM: Downloading webpage
[youtube] Wh-AgS7PKdM: Downloading tv client config
[youtube] Wh-AgS7PKdM: Downloading tv player API JSON
[youtube] Wh-AgS7PKdM: Downloading ios player API JSON


         player = https://www.youtube.com/s/player/e12fbea4/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] Wh-AgS7PKdM: Downloading m3u8 information
[info] Testing format 234
[info] Wh-AgS7PKdM: Downloading 1 format(s): 234
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 41
[download] Destination: Wh-AgS7PKdM.mp4
[download] 100% of    3.39MiB in 00:00:17 at 199.00KiB/s                


ERROR: WARNING: unable to obtain file audio codec with ffprobe


[download] Finished downloading playlist: Yellow


In [2]:
import json
songsMeta = {}
with open('Backend//songs_meta.json', 'r', encoding='utf-8') as f:
    songsMeta = json.load(f)

for song in songsMeta.values():
    song["hash_generated"] = False

with open('Backend//songs_meta.json', 'w', encoding='utf-8') as f:
    json.dump(songsMeta, f, indent=4, ensure_ascii=False)




In [7]:
import numpy as np
import librosa
from scipy.ndimage import maximum_filter, gaussian_filter

from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db=client["shazam_db"]
songs_collection=db["songs"]
hashes_collection=db["hashes"]


def SearchSong(songLocation):
    """
    Search for a song by title and artist.
    Returns the fingerprints if found, else None.
    """
    # Step 1: Load the song

    y, sr = librosa.load(songLocation, sr=None)

    # Step 2: Compute STFT
    n_fft = 2048
    hop_length = 512
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    S_mag = np.abs(S)

    # Step 3: Smooth spectrogram (optional, helps with noisy peaks)
    S_smooth = gaussian_filter(S_mag, sigma=1.0)

    # Step 4: 2D local maxima detection
    neighborhood_size = 20
    local_max = maximum_filter(S_smooth, size=neighborhood_size) == S_smooth

    # Step 5: Apply a threshold to keep only strong peaks
    threshold = np.percentile(S_smooth[local_max], 95)  # keep top 5%
    peak_mask = local_max & (S_smooth >= threshold)

    # Step 6: Get peak coordinates (freq_bin, time_bin)
    peak_indices = np.argwhere(peak_mask)

    # Step 7: Convert to time (sec) and frequency (Hz)
    peak_freqs = peak_indices[:, 0] * (sr / n_fft)
    peak_times = peak_indices[:, 1] * (hop_length / sr)

    # Combine for (time, freq) tuples
    constellation_map = list(zip(peak_times, peak_freqs))

    #get fingerprints
    fingerprint = generate_hashes(constellation_map)
    score={
    }
    for h,t_query in fingerprint:
        hashIndex = hashes_collection.find({"hash": h})
        if hashIndex:
            for doc in hashIndex:
                t_db= doc['time']
                song = doc['title']
                delta= round(t_db-t_query, 2)
                score[(song,delta)] = score.get((song,delta), 0) + 1
    best_match = None
    best_score = 0

    for (song, delta), count in score.items():
        if best_match is None or count > best_score:
            best_match = song
            best_score = count
    

    return best_match, songs_collection.find_one({"title":best_match}) if best_match else None

if __name__ =="__main__":
    songAddr="Testsongs\\Namami Shamishan (mp3cut.net).mp3"

    best_match,songsMeta=SearchSong(songAddr)
    print()
    if songsMeta:
        print(songsMeta['title'])
        print(songsMeta['artist'])
    else:
        print("No match found.")



Namami Shamishan
Religious India - Topic
